# Libraries

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from scipy import stats
import warnings

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'plotly'

# Loading Dataset

In [ ]:
data = pd.read_excel("dataset.xlsx")

In [ ]:
data.shape

# EDA

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
for col in data.columns:
    print(data[col].value_counts())

In [ ]:
data['Outcome'].value_counts()

In [ ]:
data.dtypes

In [ ]:
data['Outcome'].value_counts()/np.float(len(data))

In [ ]:
print(round(data.describe(),4))

# Feature Vector and Target Variable

In [ ]:
x=data.drop(["Outcome"],axis=1)
y=data["Outcome"]

In [ ]:
import nltk
import nltk.corpus